# <h1 align="center">The Experiment of Apriori algorithm </h1>  


**Note:** *The Assignment one of SDSC3002--Data Mining*   

**SID**: &emsp;56641800&emsp; &emsp; **Name**:&emsp; Du Junye

*This notebook is only for the use of course requirement of sdsc3002*  

## Source Code Part

In [3]:
# import relevant packages from itertools 
import os
from itertools import combinations
from itertools import chain
import time
import numpy as np 

### Read the txt file from the trans.txt
**Description:** *In this part, I discussed the influence of different data structure of the data base, the basic one is the item_set, which is a nested list of string. the transaction_index_map is a hashing dictionary, with the key of the item, the value records the number of transaction, the third one is a dictionary, with the key being the transaction, the value records the total number of the same transaction over the trans.txt to avoid calculating the same transaction repeatedly*

In [5]:
#the basic item-set
item_set = []
total = 0
with open("trans.txt", "r") as f:
    for line in f.readlines():
        total = total + 1
        line = line.split()
        item_set.append(line) 

#the hashing transaction dict
num_transaction = 0
items = []
transaction_index_map = {}
for transaction in item_set:
    for item in transaction:
        if item not in transaction_index_map:
            items.append(item)
            transaction_index_map[item] = set()
        transaction_index_map[item].add(num_transaction)
    num_transaction += 1

#the transaction dictionary    
trans = {}
for itemsss in item_set:
    tran = frozenset(itemsss)
    if tran not in trans.keys():
        trans[tran]=1
    else:
        trans[tran]+=1

**Note:** *As the loading time difference between the three data structures is too tiny to be paid attention to, so we will ignore the loading time difference.*

### Two ways to calculate the support of the according candidate  
+ <h4>The function of generate_support_dict: </h4> This function could generate a dictionary with keys the candidate, the value being the support from the whole dataset
+ <h4>The function of calc_sipport: </h4> This function calculate the support of specified candidate by virtue of the hash_index_map, with the help of the intersection between the hash-set.  

<h4>Note:</h4> As the normal version of apriori need us to enumerate the whole database everytime we calculate the support, which is time consuming disaster, so I will ignore the brute and force method and just compare the above two.

In [10]:
def generate_support_dict(length, dropped_itemset):
    hash = {}
    for items in dropped_itemset:
        keys = [frozenset(item) for item in combinations(items, length)]
        for hashkey in keys:
            if hashkey not in hash:
                hash[hashkey] = 0
            hash[hashkey] += 1
    return hash

In [11]:
def calc_support(items):
    sum_indexs = None 
    for item in items:
        indexes = transaction_index_map.get(item)
        if sum_indexs is None:
            sum_indexs = indexes
        sum_indexs = sum_indexs.intersection(indexes)
    return len(sum_indexs)

### Two functions of generating the candidates 
+ Generating method including prunning 
+ Generating method without prunning 

In [6]:
def create_next_candidates(prev_candidates, length):
    #items = sorted(frozenset(chain.from_iterable(prev_candidates)))
    #tmp_next_candidates = (frozenset(x) for x in combinations(items, length))
    
    tmp_next_candidates = set([i.union(j) for i in prev_candidates for j in prev_candidates if len(i.union(j)) == length])
    if length < 3:
        return list(tmp_next_candidates)
    next_candidates = [candidate for candidate in tmp_next_candidates if all(frozenset(x) in prev_candidates for x in combinations(candidate, length - 1))]
    return next_candidates


In [7]:
def Candidate_Generation(prev_freq_items,k):
    return set([i.union(j) for i in prev_freq_items for j in prev_freq_items if len(i.union(j)) == k])

### Basic Apriori algorithm without ignoring transations without any frequent (k-1)items

In [8]:
def gen_support_records4(item_set, min_support, items):
    relations = set()
    countk = {}
    candidates = [frozenset([item]) for item in items]
    length = 1
    dropped_itemset = item_set
    while candidates:
        num = 0

        #hashstart = time.time()
        hash = generate_support_dict(length,dropped_itemset)
        #hashend = time.time()
        #print("Length {} takes hashtime {}".format(length, hashend- hashstart))

        relations = set()
        for relation_candidate in candidates:
            #print(relation_candidate)
            support = hash.get(relation_candidate)
            #print(support)
            if (support is None) or (support < min_support):
                continue 
            num += 1
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
        countk[length] = num     
        length += 1       

        #generatestart = time.time()
        candidates = create_next_candidates(relations, length)
        #generateend = time.time()
        #print("The generating time of length {} is {}".format(length, generateend - generatestart))
    print(countk)

In [39]:
aaa = time.time()
for i in np.linspace(0.0005,0.0001,5):
    gen_support_records4(item_set, total*i, items)
bbb = time.time()
print("The total time is {}s".format(bbb-aaa))

{1: 437, 2: 1727, 3: 505, 4: 26, 5: 0}
{1: 470, 2: 2237, 3: 800, 4: 49, 5: 0}
{1: 524, 2: 3081, 3: 1342, 4: 129, 5: 1}
{1: 592, 2: 4553, 3: 2734, 4: 334, 5: 6}
{1: 733, 2: 8133, 3: 7998, 4: 1740, 5: 90, 6: 0}
The total time is 286.5076746940613s


### The Apriori algorithm without prunning operation 

In [ ]:
def gen_support_records5(item_set, min_support, items):
    #print("  ")
    #print("The following is the {}".format(min_support/total))
    relations = set()
    countk = {}
    candidates = [frozenset([item]) for item in items]
    length = 1
    dropped_itemset = item_set
    while candidates:
        num = 0
        #print(relations)
        #dropstart = time.time()
        if length>=2:
            dropped_itemset = [tran for tran in dropped_itemset if any(frozenset(x) in relations for x in combinations(tran, length - 1))]
        #dropend = time.time()
        #print("Length {} takes dropping time is {} ".format(length, dropend-dropstart))

        #hashstart = time.time()
        hash = generate_support_dict(length,dropped_itemset)
        #hashend = time.time()
        #print("Length {} takes hashtime {}".format(length, hashend- hashstart))
        relations = set()
        for relation_candidate in candidates:
            #print(relation_candidate)
            support = hash.get(relation_candidate)
            #print(support)
            if (support is None) or (support < min_support):
                continue 
            num += 1
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
        countk[length] = num     
        length += 1       
        #generatestart = time.time()
        candidates = Candidate_Generation(relations, length)
        #generateend = time.time()
        #print("The generating time of length {} is {}".format(length, generateend - generatestart))

    print(countk)

In [36]:
aaa = time.time()
for i in np.linspace(0.0005,0.0001,5):
    gen_support_records5(item_set, total*i, items)
bbb = time.time()
print("The total time is {}s".format(bbb-aaa))

{1: 437, 2: 1727, 3: 505, 4: 26, 5: 0}
{1: 470, 2: 2237, 3: 800, 4: 49, 5: 0}
{1: 524, 2: 3081, 3: 1342, 4: 129, 5: 1}
{1: 592, 2: 4553, 3: 2734, 4: 334, 5: 6, 6: 0}
{1: 733, 2: 8133, 3: 7998, 4: 1740, 5: 90, 6: 0}
The total time is 211.2333381175995s


### Apriori algorithm with ignoring transations without any frequent (k-1)items

In [21]:
# drop的apriori
def gen_support_records2(item_set, min_support, items):
    relations = set()
    countk = {}
    candidates = [frozenset([item]) for item in items]
    length = 1
    dropped_itemset = item_set
    while candidates:
        num = 0
        #print(relations)
        #dropstart = time.time()
        if length>=2:
            dropped_itemset = [tran for tran in dropped_itemset if any(frozenset(x) in relations for x in combinations(tran, length - 1))]
        #dropend = time.time()
        #print("Length {} takes dropping time is {} ".format(length, dropend-dropstart))

        #hashstart = time.time()
        hash = generate_support_dict(length,dropped_itemset)
        #hashend = time.time()
        #print("Length {} takes hashtime {}".format(length, hashend- hashstart))
        
        relations = set()
        for relation_candidate in candidates:
            #print(relation_candidate)
            support = hash.get(relation_candidate)
            #print(support)
            if (support is None) or (support < min_support):
                continue 
            num += 1
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
        countk[length] = num     
        length += 1       

        #generatestart = time.time()
        candidates = create_next_candidates(relations, length)
        #generateend = time.time()
        #print("The generating time of length {} is {}".format(length, generateend - generatestart))

    print(countk)

In [38]:
aaa = time.time()
for i in np.linspace(0.0005,0.0001,5):
    gen_support_records2(item_set, total*i, items)
bbb = time.time()
print("The total time is {}s".format(bbb-aaa))

{1: 437, 2: 1727, 3: 505, 4: 26, 5: 0}
{1: 470, 2: 2237, 3: 800, 4: 49, 5: 0}
{1: 524, 2: 3081, 3: 1342, 4: 129, 5: 1}
{1: 592, 2: 4553, 3: 2734, 4: 334, 5: 6}
{1: 733, 2: 8133, 3: 7998, 4: 1740, 5: 90, 6: 0}
The total time is 207.4671196937561s


### The apriori with hash_map_index data structure

In [15]:
def gen_support_records(min_support, items):
    countk = {}
    candidates = [frozenset([item]) for item in items]
    length = 1
    while candidates:
        num = 0
        relations = set()

        start = time.time()
        for relation_candidate in candidates:

            support = calc_support(relation_candidate)

            if support < min_support:
                continue
            num += 1
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)

        end = time.time()
        #print("The length {} support calculating time is {}".format(length, end-start))
        
        countk[length] = num     
        length += 1       

        #generatestart = time.time()
        candidates = create_next_candidates(relations, length)
        #generateend = time.time()
        #print("The generating time of length {} is {}".format(length, generateend - generatestart))

    print(countk)

In [41]:
aaa = time.time()
for i in np.linspace(0.0005,0.0001,5):
    gen_support_records(total*i, items)
bbb = time.time()
print("The total time is {}s".format(bbb-aaa))

{1: 437, 2: 1727, 3: 505, 4: 26, 5: 0}
{1: 470, 2: 2237, 3: 800, 4: 49, 5: 0}
{1: 524, 2: 3081, 3: 1342, 4: 129, 5: 1}
{1: 592, 2: 4553, 3: 2734, 4: 334, 5: 6}
{1: 733, 2: 8133, 3: 7998, 4: 1740, 5: 90, 6: 0}
The total time is 375.70855164527893s


### The apriori using dictionary data sturcture

In [42]:
def generate_support_dict(length, dropped_dict):
    hash = {}
    for items in dropped_dict:

        num = dropped_dict.get(items)
        keys = [frozenset(item) for item in combinations(items, length)]
        
        for hashkey in keys:
            if hashkey not in hash:
                hash[hashkey] = 0
            hash[hashkey] += num 
    return hash

def gen_support_records3(trans, min_support):
    countk = {}
    candidates = [frozenset([item]) for item in items]
    #print(candidates)
    length = 1
    dropped_trans = trans
    while candidates:
        num = 0
        
        if length>=2:
            dropped_trans = {k:v for k,v in dropped_trans.items() if any(frozenset(x) in relations for x in combinations(k, length - 1))}

        hash_trans = generate_support_dict(length, dropped_trans)
        
        relations = set()
        
        for relation_candidate in candidates:
            #print(relation_candidate)

            support = hash_trans.get(relation_candidate)
            
            #print(support)
            if (support == None) or (support < min_support):
                continue
            num += 1
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
        countk[length] = num     
        length += 1       
        candidates = create_next_candidates(relations, length)
    print(countk)

In [52]:
aaa = time.time()
for i in np.linspace(0.0005,0.0001,5):
    gen_support_records3(trans, total*i)
bbb = time.time()
print("The total time is {}s".format(bbb-aaa))

{1: 437, 2: 1727, 3: 505, 4: 26, 5: 0}
{1: 470, 2: 2237, 3: 800, 4: 49, 5: 0}
{1: 524, 2: 3081, 3: 1342, 4: 129, 5: 1}
{1: 592, 2: 4553, 3: 2734, 4: 334, 5: 6}
{1: 733, 2: 8133, 3: 7998, 4: 1740, 5: 90, 6: 0}
The total time is 171.0808961391449s


## Result Conclusion Part  
*From the experiment result above, my code could achieve the goal with the fastest speed of **171s**, less than 3 minutes. In the following conetent I will present the comparison between different methods and data structures.*

### Comparison betweeen the different method of calculating support

**Note:** *From the above experiment we could see that the hashing dictionary method's efficiency is much higher than the hash_map_index method, for in the hashing dictionary method, the program only need to enumerate the whole transaction once and build a hashing table so that every items' support could be directely got from the dict, but the set intersection method requires it to calculate the number of intersection elements for every candidate. But the advantage of intersectin method is that, when the number of candiates is not so large, it could has a much better performance.*  

**The Hashing Dictionary Support Calculating Method:**

| min_support | k = 1  | k = 2 | k = 3  | k = 4 | k = 5 |
| ----------- | ------ | ----- | ------ | ----- | ----- |
| 0.0005      | 0.93   | 2.25  | 5.97   | 7.53  | 2.65  |
| 0.0004      | 0.93   | 2.35  | 5.61   | 8.04  | 3.11  |
| 0.0003      | 0.94   | 2.30  | 6.39   | 8.80  | 4.01  |
| 0.0002      | 0.91   | 2.29  | 6.33   | 9.10  | 6.54  |
| 0.0001      | 0.91   | 2.18  | 5.48   | 9.46  | 11.18 |  


**The Intersection of Set Method:** 

| min_support | k = 1  | k = 2 | k = 3  | k = 4 | k = 5 |
| ----------- | ------ | ----- | ------ | ----- | ----- |
| 0.0005      | 0.008  | 17.68 | 14.56  | 0.76  | 0.003 |
| 0.0004      | 0.007  | 19.10 | 19.64  | 1.27  | 0.001 |
| 0.0003      | 0.008  | 20.60 | 28.40  | 2.49  | 0.05  |
| 0.0002      | 0.006  | 23.01 | 46.69  | 6.05  | 0.22  |
| 0.0001      | 0.006  | 27.73 | 94.06  | 27.67 | 1.35  |



### Comparison betweeen the prunning operation 

**Note:** *From the experiment above, we could find that the running time of the program decrease from 211s to 207s, which is not so obvious according to the total running time, I ascribe it to the hashing dictionary of candidate support as in the normal Apriori, we need to enumerate the dataset every time we need to calculate the support, but since we only need to build a dict once, so the serching time would not make a large difference.*

### Comparison between the ignoring operation

**Note:** *The Ignoring Operation makes a huge difference in the experiment, which cut down some items every iteration, the operation could decrease time in both building hashing dictionary and candidate generating. From the experiment, we could see an improvement from 
286s to 207s, with around 35% improvement.*

### The Program with best performance

In my program the model with best performance used the dictionary data sturcture, which loaded every transaction as the key of a dictionary, this kind of data structure saved time in computing duplicate to a large extent. 

In [20]:
trans 

{frozenset({'0', '1', '2'}): 1,
 frozenset({'3'}): 15075,
 frozenset({'3', '4'}): 39,
 frozenset({'0', '10', '5', '6', '7', '8', '9'}): 1,
 frozenset({'11', '12'}): 4,
 frozenset({'13', '14', '15'}): 1,
 frozenset({'15', '16', '17', '18', '19'}): 1,
 frozenset({'1', '20', '21', '22', '23'}): 1,
 frozenset({'13', '24', '25', '6'}): 1,
 frozenset({'19', '22', '23', '26', '27'}): 1,
 frozenset({'19', '28', '29'}): 2,
 frozenset({'19'}): 3484,
 frozenset({'19', '21'}): 101,
 frozenset({'17', '19', '24', '30', '31', '32', '7'}): 1,
 frozenset({'21', '33'}): 258,
 frozenset({'1', '34', '35'}): 21,
 frozenset({'36', '37'}): 4,
 frozenset({'1', '21'}): 6418,
 frozenset({'1'}): 26669,
 frozenset({'21', '38'}): 22,
 frozenset({'39', '40'}): 43,
 frozenset({'0', '1', '13', '15', '38', '41', '42', '43'}): 1,
 frozenset({'0', '10'}): 2,
 frozenset({'1', '11'}): 30,
 frozenset({'0', '19', '44', '45', '46', '47'}): 1,
 frozenset({'15', '17', '24', '30', '48', '49', '50', '51', '52', '53'}): 1,
 froze

As we could see above,the transaction {'3'} has 15075 duplicate.This kind of data structure has a much better performance when it comes to the duplicate transactions. 